In [11]:
import io, os, sys, types
from IPython import get_ipython
from IPython.nbformat import current
from IPython.core.interactiveshell import InteractiveShell

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.worksheets[0].cells:
            if cell.cell_type == 'code' and cell.language == 'python':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.input)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
sys.meta_path.append(NotebookFinder())

In [14]:
from model import ConvWTA
import tensorflow as tf
# tf.set_random_seed(2017)
tf.random.set_seed(2017)

import os
ckpt_dir = "ckpt/"
if not os.path.isdir(ckpt_dir): 
  os.makedirs(ckpt_dir)
ckpt_path = "ckpt/model.ckpt"

epochs = 100
batch_size = 100
learning_rate = 1e-3
shape = [batch_size, 28, 28, 1]

# Basic tensorflow setting
sess = tf.Session()
ae = ConvWTA(sess)
x = tf.placeholder(tf.float32, shape)
loss = ae.loss(x, lifetime_sparsity=0.05)

optim = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optim.minimize(loss, var_list=ae.t_vars)

sess.run(tf.global_variables_initializer())

# Data read & train
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist/", one_hot=True)

import time
start_time = time.time()
for epoch in range(epochs):
  total_batch = int(mnist.train.num_examples / batch_size)
  avg_loss = 0
  for i in range(total_batch):
    batch_x, _ = mnist.train.next_batch(batch_size)

    batch_x = batch_x.reshape(shape)
    l, _ =  sess.run([loss, train], {x:batch_x})
    avg_loss += l / total_batch
    
  print("Epoch : {:04d}, Loss : {:.9f}".format(epoch+1, avg_loss))
print("Training time : {}".format(time.time() - start_time))

ae.save(ckpt_path)

AttributeError: module 'tensorflow' has no attribute 'variable_scope'